# Segmenting and Clustering Neighborhoods in the city of Toronto, Canada
### Applied Data Science Capstone - Week 3 Assignmnet
#### w. Eunkyung Gu

### Installing Libraries

In [1]:
# Install

#!pip install lxml html5lib beautifulsoup4
!pip install folium
#!pip install geocoder
#!conda install -c conda-forge geopy --yes

print("Install finished!")

import pandas as pd
import numpy as np

import folium

from geopy.geocoders import Nominatim

import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans 

import json
import requests
from pandas.io.json import json_normalize

print("Import finished!")

Install finished!
Import finished!


### 1. Pandas Web Scraping
Get the website into DataFrame from Wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
df.shape

(180, 3)

### 2. Process the data
1. Drop the row where the Borough is 'Not assigned'
2. If few neighborhoods exist in one postal code, they will be combined into one row.
3. Neighborhood will be the same as the borough if neighborhood is 'Not assigned'
4. Markdown cells to explain my work and assumptions I made
5. In the last cell, use the `.shape ` method to print the number of rows

In [4]:
# Drop the row where the Borough is 'Not assigned'

idx_1 = df[df['Borough'] == 'Not assigned'].index
df = df.drop(idx_1)
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
df.shape

(103, 3)

I didn't do second stage because they already are combined into one row.

In [6]:
# Change the neighbourhood 'Not assigned' into the same as the borough

idx_2 = df[df['Neighbourhood'] == 'Not assigned'].index
df.iloc[idx_2, 2] = df.iloc[idx_2,1]


In [7]:
df.shape

(103, 3)

### 3. Get the latitude and longitude coordinates of each neighborhood.
Merge csv file with dataframe.

In [8]:
"""
Below code is not working(Geocoder denied),
So I will get lat and lng from csv file.

import geocoder

for i in range(0, df.shape[0]):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(df.loc[i, 'Postal Code']))
        lat_lng_coords = g.latlng
        df.loc[i, 'Latitude'] = lat_lng_coords[0]
        df.loc[i, 'Longitude'] = lat_lng_coords[1]

df.head()

"""

"\nBelow code is not working(Geocoder denied),\nSo I will get lat and lng from csv file.\n\nimport geocoder\n\nfor i in range(0, df.shape[0]):\n    lat_lng_coords = None\n    while(lat_lng_coords is None):\n        g = geocoder.google('{}, Toronto, Ontario'.format(df.loc[i, 'Postal Code']))\n        lat_lng_coords = g.latlng\n        df.loc[i, 'Latitude'] = lat_lng_coords[0]\n        df.loc[i, 'Longitude'] = lat_lng_coords[1]\n\ndf.head()\n\n"

In [9]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Merge the Dataframe

df_merged = df
df_merged = df.join(df_data_1.set_index('Postal Code'),
                    on = 'Postal Code')
df_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [11]:
print('The dataframe has {} boroughs.'.format(len(df['Borough'].unique())))

The dataframe has 10 boroughs.


DataFrame is prepared!

### 4. Visualization on maps
Create a map of Toronto and Superimpose neighborhoods

In [12]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347


In [13]:
# create map of toronto
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'blue'
    ).add_to(map_toronto)

map_toronto

#### EX) Downtown Toronto
Make a map and neighbourhood of "Downtown Toronto" Borough

In [14]:
dt_data = df_merged[df_merged['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dt_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [15]:
dt_data.shape

(19, 5)

In [16]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent = 'toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geopgrapical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geopgrapical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [17]:
map_dt = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat, lng, label in zip(dt_data['Latitude'], dt_data['Longitude'], dt_data['Neighbourhood']):
    labe = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'blue',
    ).add_to(map_dt)
    
map_dt

### 5. Using Foursquare, Get top 100 venues

Define Foursquare Credentials and Version.

In [18]:
CLIENT_ID = 'VR3MWVMQV1U0RGG0S223ZPXPJC21KBF4CUFQ3FVMGUNBDSCE' # your Foursquare ID
CLIENT_SECRET = '3TDSWJFLHBMYRUSM0QS3I0XYFNSROLOLJVVDR3HFGX2HYIY3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VR3MWVMQV1U0RGG0S223ZPXPJC21KBF4CUFQ3FVMGUNBDSCE
CLIENT_SECRET:3TDSWJFLHBMYRUSM0QS3I0XYFNSROLOLJVVDR3HFGX2HYIY3


#### EX) Regent Park, Harbourfront
There is "Regent Park, Harbourfront" Neighbourhood in "Downtown Toronto" Borough. I will get 100 venues for Regent Park, Harbourfront.

In [19]:
dt_data.loc[0,'Neighbourhood']

'Regent Park, Harbourfront'

In [20]:
neighbourhood_latitude = dt_data.loc[0, 'Latitude']
neighbourhood_longitude = dt_data.loc[0, 'Longitude']

neighbourhood_name = dt_data.loc[0, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name,
                                                              neighbourhood_latitude,
                                                              neighbourhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [21]:
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, neighbourhood_latitude, neighbourhood_longitude, radius, LIMIT
)
url

'https://api.foursquare.com/v2/venues/explore?client_id=VR3MWVMQV1U0RGG0S223ZPXPJC21KBF4CUFQ3FVMGUNBDSCE&client_secret=3TDSWJFLHBMYRUSM0QS3I0XYFNSROLOLJVVDR3HFGX2HYIY3&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc9bc0b9689f0727e74b2ab'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [25]:
print('{} venues wer returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues wer returned by Foursquare.


### 6. Explore Neighborhoods in Toronto
For every Neighbourhood in Downtown Toronto Borough, I will get top 100 venues.

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                            'Neighbourhood Latitude',
                            'Neighbourhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']
    
    return(nearby_venues)

In [27]:
dt_venues = getNearbyVenues(names = dt_data['Neighbourhood'], latitudes = dt_data['Latitude'], longitudes = dt_data['Longitude'], radius=500)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [28]:
print(dt_venues.shape)
dt_venues.head()

(1248, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [29]:
dt_venues.groupby('Neighbourhood').count()


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,55,55,55,55,55,55
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,68,68,68,68,68,68
Christie,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [31]:
print('There are {} uniques categories.'.format(len(dt_venues['Venue Category'].unique())))

There are 210 uniques categories.


#### Analyze Each Neighborhood
Analyze category and its frquency of venue for each Neighbourhood.

In [32]:
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix = "", prefix_sep = "")

dt_onehot['Neighbourhood'] = dt_venues['Neighbourhood']

fixed_columns = [dt_onehot.columns[-1]]+list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix = "", prefix_sep = "")

dt_onehot['Neighbourhood'] = dt_venues['Neighbourhood']

fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]
dt_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
dt_onehot.shape

(1248, 211)

In [35]:
dt_grouped = dt_onehot.groupby('Neighbourhood').mean().reset_index()
dt_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.018182,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.014706,0.000000,0.000000,0.014706,0.014706
3,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.013333,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.013333,0.000000,...,0.013333,0.013333,0.00,0.00,0.00,0.000000,0.000000,0.013333,0.000000,0.026667
5,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.030000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.030000,0.000000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000
7,"Garden District, Ryerson",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.020000,0.000000,0.01,0.00,0.00,0.000000,0.010000,0.010000,0.010000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.010000,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.054054,0.000000,0.040541,0.013514,0.000000


In [36]:
dt_grouped.shape

(19, 211)

Choose top 5 frequent venues for each Neighbourhood.

In [37]:
num_top_venues = 5

for hood in dt_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.04
2            Beer Bar  0.04
3      Farmers Market  0.04
4  Seafood Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0   Airport Lounge  0.12
1  Airport Service  0.12
2  Harbor / Marina  0.06
3          Airport  0.06
4         Boutique  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1                Café  0.06
2      Sandwich Place  0.04
3  Italian Restaurant  0.04
4     Thai Restaurant  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3     Restaurant  0.06
4     Baby Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1  Japanese Restaurant  0.05
2     Sushi Restaurant  0.05
3              Gay Bar  0.05
4           Res

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighbourhoods_venues_sorted = pd.DataFrame(columns = columns)
neighbourhoods_venues_sorted['Neighbourhood'] = dt_grouped['Neighbourhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)
    
neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cheese Shop,Farmers Market,Cocktail Bar,Restaurant,Seafood Restaurant,Bakery,Beer Bar,Gourmet Shop,Basketball Stadium
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boat or Ferry,Boutique,Plane,Rental Car Location,Coffee Shop,Harbor / Marina,Sculpture Garden,Airport Terminal
2,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Salad Place,Japanese Restaurant,Department Store,Thai Restaurant,Burger Joint
3,Christie,Grocery Store,Café,Park,Candy Store,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Nightclub,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Pub
5,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Italian Restaurant,Deli / Bodega,Seafood Restaurant,Japanese Restaurant
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,American Restaurant,Asian Restaurant,Steakhouse,Salad Place
7,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Ramen Restaurant,Pizza Place,Bookstore,Middle Eastern Restaurant
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Restaurant,Fried Chicken Joint,Pizza Place,Brewery,History Museum
9,"Kensington Market, Chinatown, Grange Park",Mexican Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Café,Bar,Vietnamese Restaurant,Grocery Store,Dessert Shop,Park,Pizza Place


### 7. Cluster Neighborhoods
I will devide Neighborhoods into 5 clusters using K-means Clustering.

In [41]:
# number of clusters
kclusters = 5

dt_grouped_clustering = dt_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(dt_grouped_clustering)

kmeans.labels_

array([2, 3, 2, 4, 2, 2, 2, 2, 2, 2, 0, 0, 2, 1, 2, 2, 2, 2, 2],
      dtype=int32)

In [42]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [43]:
dt_merged = dt_data

dt_merged = dt_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'),
                           on = 'Neighbourhood')

dt_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Yoga Studio,Event Space,Performing Arts Venue
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Portuguese Restaurant,Smoothie Shop,Italian Restaurant,Beer Bar,Sandwich Place,Restaurant,Distribution Center,Diner
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Ramen Restaurant,Pizza Place,Bookstore,Middle Eastern Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub,American Restaurant,Beer Bar,Seafood Restaurant,Gym,Farmers Market
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cheese Shop,Farmers Market,Cocktail Bar,Restaurant,Seafood Restaurant,Bakery,Beer Bar,Gourmet Shop,Basketball Stadium


In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters) # kclusters = 5
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighbourhood'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

In [45]:
# Make list including Neighbourhood and Common Venues

fix_list = list(range(5, dt_merged.shape[1]))
fix_list.insert(0, 2)

#### Cluster 1

In [46]:
dt_merged.loc[dt_merged['Cluster Labels'] == 0, dt_merged.columns[fix_list]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Yoga Studio,Event Space,Performing Arts Venue
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Yoga Studio,Portuguese Restaurant,Smoothie Shop,Italian Restaurant,Beer Bar,Sandwich Place,Restaurant,Distribution Center,Diner


#### Cluster 2

In [47]:
dt_merged.loc[dt_merged['Cluster Labels'] == 1, dt_merged.columns[fix_list]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,1,Park,Trail,Playground,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


#### Cluster 3

In [48]:
dt_merged.loc[dt_merged['Cluster Labels'] == 2, dt_merged.columns[fix_list]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Garden District, Ryerson",2,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Ramen Restaurant,Pizza Place,Bookstore,Middle Eastern Restaurant
3,St. James Town,2,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub,American Restaurant,Beer Bar,Seafood Restaurant,Gym,Farmers Market
4,Berczy Park,2,Coffee Shop,Cheese Shop,Farmers Market,Cocktail Bar,Restaurant,Seafood Restaurant,Bakery,Beer Bar,Gourmet Shop,Basketball Stadium
5,Central Bay Street,2,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Salad Place,Japanese Restaurant,Department Store,Thai Restaurant,Burger Joint
7,"Richmond, Adelaide, King",2,Coffee Shop,Café,Hotel,Gym,Restaurant,Thai Restaurant,Bar,Clothing Store,Juice Bar,Concert Hall
8,"Harbourfront East, Union Station, Toronto Islands",2,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Restaurant,Fried Chicken Joint,Pizza Place,Brewery,History Museum
9,"Toronto Dominion Centre, Design Exchange",2,Coffee Shop,Hotel,Restaurant,Café,Salad Place,Seafood Restaurant,American Restaurant,Japanese Restaurant,Sporting Goods Shop,Italian Restaurant
10,"Commerce Court, Victoria Hotel",2,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Italian Restaurant,Deli / Bodega,Seafood Restaurant,Japanese Restaurant
11,"University of Toronto, Harbord",2,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Bakery,Yoga Studio,Italian Restaurant,Beer Bar,Beer Store
12,"Kensington Market, Chinatown, Grange Park",2,Mexican Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Café,Bar,Vietnamese Restaurant,Grocery Store,Dessert Shop,Park,Pizza Place


#### Cluster 4

In [49]:
dt_merged.loc[dt_merged['Cluster Labels'] == 3, dt_merged.columns[fix_list]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",3,Airport Lounge,Airport Service,Boat or Ferry,Boutique,Plane,Rental Car Location,Coffee Shop,Harbor / Marina,Sculpture Garden,Airport Terminal


#### Cluster 5

In [50]:
dt_merged.loc[dt_merged['Cluster Labels'] == 4, dt_merged.columns[fix_list]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,4,Grocery Store,Café,Park,Candy Store,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Nightclub,Coffee Shop
